In [1]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = "/Users/duhameleve/Desktop/M2_SNS/M2SNS/Sante/Github/HAH913E-2024-projet/AX3 Project/Sans titre/data/NoneSmokerCardio.csv"

data = pd.read_csv(file_path)

timestamp_column = data.columns[0]
signal_columns = data.columns[1:]   # All remaining columns are sensor data

# Convert the timestamp column to datetime
data[timestamp_column] = pd.to_datetime(data[timestamp_column], errors='coerce')  # Handle invalid dates

# Calculate the sampling interval (in seconds)
sampling_interval = (
    pd.to_datetime(data[timestamp_column].iloc[1]) -  # Access a single row
    pd.to_datetime(data[timestamp_column].iloc[0])
).total_seconds()

# Create a relative time axis (in minutes)
data['relative_time_minutes'] = data.index * sampling_interval

# Define axis labels for clarity
axis_labels = ['X-axis', 'Y-axis', 'Z-axis']
colors = {'X': 'blue', 'Y': 'red', 'Z': 'green'}  # Define colors for each signal

In [2]:
calibration_offset = {
    '-0.296875': -1.62,   # Offset pour l'axe X
    '0.703125': -0.88,    # Offset pour l'axe Y
    '-0.343750': -1.16    # Offset pour l'axe Z
}
for col, offset in calibration_offset.items():
    data[col] = data[col] - offset

from scipy.signal import butter, filtfilt

# Function to apply a low-pass filter
def lowpass_filter(data, cutoff, fs, order=4):
    """
    Apply a low-pass Butterworth filter.
    
    Parameters:
        data (array-like): The signal to filter.
        cutoff (float): The cutoff frequency in Hz.
        fs (float): The sampling frequency in Hz.
        order (int): The order of the filter (default is 4).
    
    Returns:
        array-like: The filtered signal.
    """
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data)

fs = 1 / sampling_interval  # Fréquence d'échantillonnage en Hz
cutoff = 2  # Fréquence de coupure en Hz
for col in signal_columns:
    data[col] = lowpass_filter(data[col], cutoff, fs)

KeyError: '-0.296875'

In [ ]:
import numpy as np

# Calcul de la norme euclidienne des accélérations (X, Y, Z)
data.loc[:, 'norm'] = np.sqrt(data['-0.296875']**2 + data['0.703125']**2 + data['-0.343750']**2)

# Normalisation : centrer et réduire les données
mean_norm = data['norm'].mean()
std_norm = data['norm'].std()
data.loc[:, 'norm_normalized'] = (data['norm'] - mean_norm) / std_norm

# Calcul de l'ENMO : norme normalisée - 1, et on garde la valeur maximale entre 0 et la norme
data.loc[:, 'ENMO'] = np.maximum(data['norm_normalized'] - 1, 0)

# Affichage des premiers résultats
print(data[['relative_time_minutes', 'ENMO']].head())

# Visualisation de l'ENMO normalisé
plt.figure(figsize=(12, 6))
plt.plot(data['relative_time_minutes'], data['ENMO'], label="ENMO normalisé", color='purple')
plt.title("ENMO au fil du temps (Normalisé)")
plt.xlabel("Temps (min)")
plt.ylabel("ENMO")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Définir les seuils pour les intensités
seuil_faible = 0.5
seuil_intense = 1.5

# Classifier les moments d'intensité
moments_faibles = data[data['ENMO'] <= seuil_faible]
moments_moderes = data[(data['ENMO'] > seuil_faible) & (data['ENMO'] <= seuil_intense)]
moments_intenses = data[data['ENMO'] > seuil_intense]

# Visualisation des intensités
plt.figure(figsize=(12, 6))
plt.plot(data['relative_time_minutes'], data['ENMO'], label="ENMO", color='purple')
plt.axhline(y=seuil_faible, color='blue', linestyle='--', label="Seuil d'activité faible (0.5)")
plt.axhline(y=seuil_intense, color='red', linestyle='--', label="Seuil d'activité intense (1.5)")
plt.title("Différenciation des Intensités : Faible, Modérée et Intense")
plt.xlabel("Temps (min)")
plt.ylabel("ENMO")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Statistiques de base
stats = data['ENMO'].describe()
print(stats)